# 네이버 주식 크롤링

In [47]:
import pandas as pd
from pandas import DataFrame, Series
import requests as re
from bs4 import BeautifulSoup
import datetime as date
import time
import traceback

In [55]:
# 주식코드 입력 및 마지막 페이지 확인
code = '005930'
url = 'http://finance.naver.com/item/sise_day.nhn?code={code}'.format(code=code)
res = re.get(url)
res.encoding = 'utf-8'
res.status_code
soap = BeautifulSoup(res.text, 'lxml')
el_table_navi = soap.find("table", class_="Nnavi")
el_td_last = el_table_navi.find("td", class_="pgRR")
pg_last = el_td_last.a.get('href').rsplit('&')[1]
pg_last = pg_last.split('=')[1]
pg_last = int(pg_last)
pg_last

609

In [56]:
def parse_page(code, page):
    try:
        url = 'http://finance.naver.com/item/sise_day.nhn?code={code}&page={page}'.format(code=code, page=page)
        res = requests.get(url)
        _soap = BeautifulSoup(res.text, 'lxml')
        _df = pd.read_html(str(_soap.find("table")), header=0)[0]
        _df = _df.dropna()
        return _df
    except Exception as e:
        traceback.print_exc()
    return None
parse_page(code, 1)

,날짜,종가,전일비,시가,고가,저가,거래량
1,2020.09.02,54500.0,300.0,54600.0,55100.0,54400.0,7021845.0
2,2020.09.01,54200.0,200.0,54100.0,54800.0,54100.0,19363117.0
3,2020.08.31,54000.0,1400.0,56000.0,56100.0,54000.0,32671367.0
4,2020.08.28,55400.0,200.0,56100.0,56300.0,55400.0,14619888.0
5,2020.08.27,55600.0,800.0,56300.0,56300.0,55600.0,16196568.0
9,2020.08.26,56400.0,0.0,56400.0,56500.0,55700.0,17651593.0
10,2020.08.25,56400.0,300.0,56400.0,56800.0,56100.0,14021705.0
11,2020.08.24,56100.0,200.0,55800.0,56600.0,55400.0,15055896.0
12,2020.08.21,55900.0,500.0,56200.0,56900.0,55800.0,21142288.0
13,2020.08.20,55400.0,2400.0,57600.0,57600.0,55300.0,30386029.0


In [57]:
# 이 날짜 이후로 크롤링
str_datefrom = date.datetime.strftime(date.datetime(year=2017, month=8, day=1), '%Y.%m.%d')
str_datefrom

'2017.08.01'

In [58]:
# 현재 날짜 확인
str_dateto = date.datetime.strftime(date.datetime.today(), '%Y.%m.%d')
str_dateto

'2020.09.02'

In [59]:
df = None
for page in range(1, pg_last+1):
    _df = parse_page(code, page)
    _df_filtered = _df[_df['날짜'] > str_datefrom]
    if df is None:
        df = _df_filtered
    else:
        df = pd.concat([df, _df_filtered])
    if len(_df) > len(_df_filtered):
        break
df.head()

,날짜,종가,전일비,시가,고가,저가,거래량
1,2020.09.02,54500.0,300.0,54600.0,55100.0,54400.0,7021845.0
2,2020.09.01,54200.0,200.0,54100.0,54800.0,54100.0,19363117.0
3,2020.08.31,54000.0,1400.0,56000.0,56100.0,54000.0,32671367.0
4,2020.08.28,55400.0,200.0,56100.0,56300.0,55400.0,14619888.0
5,2020.08.27,55600.0,800.0,56300.0,56300.0,55600.0,16196568.0


In [53]:
import os
path_dir = 'data/naver_crawling'
if not os.path.exists(path_dir):
    os.makedirs(path_dir)
path = os.path.join(path_dir, '{code}_{date_from}_{date_to}.csv'.format(code=code, date_from=str_datefrom, date_to=str_dateto))
path

'data/naver_crawling\\005930_2017.08.01_2020.09.02.csv'

In [54]:
df.to_csv(path, index=False)

# 네이버 금값시세 크롤링

In [12]:
folder_adress = 'D:\Workspace\Deep-Learning\data'
def market_index_crawling():
    Data = DataFrame()
    
    url_dict = {'국제 금':'http://finance.naver.com/marketindex/worldDailyQuote.nhn?fdtc=2&marketindexCd=CMDT_CDY&'}

    for key in url_dict.keys():
    
        date = []
        value = []

        for i in range(1,1000):
            url = re.get(url_dict[key] + '&page=%s'%i)
            url = url.content

            html = BeautifulSoup(url,'html.parser')

            tbody = html.find('tbody')
            tr = tbody.find_all('tr')
            
            
            '''마지막 페이지 까지 받기'''
            if len(tbody.text.strip()) > 3:
                
                for r in tr:
                    temp_date = r.find('td',{'class':'date'}).text.replace('.','-').strip()
                    temp_value = r.find('td',{'class':'num'}).text.strip()
            
                    date.append(temp_date)
                    value.append(temp_value)
            else:

                temp = DataFrame(value, index = date, columns = [key])
                
                Data = pd.merge(Data,temp, how='outer', left_index=True, right_index=True)        
                
                print(key + '자료 수집 완료')
                time.sleep(10)
                break

    Data.to_csv('%s/gold_market_index.csv'%(folder_adress))
    return Data

In [9]:
def market_index_crawling():
    Data = DataFrame()
    
    url_dict = {'국제 은':'https://finance.naver.com/marketindex/worldDailyQuote.nhn?marketindexCd=CMDT_SI&fdtc=2'}

    for key in url_dict.keys():
    
        date = []
        value = []

        for i in range(1,1000):
            url = re.get(url_dict[key] + '&page=%s'%i)
            url = url.content

            html = BeautifulSoup(url,'html.parser')

            tbody = html.find('tbody')
            tr = tbody.find_all('tr')
            
            
            '''마지막 페이지 까지 받기'''
            if len(tbody.text.strip()) > 3:
                
                for r in tr:
                    temp_date = r.find('td',{'class':'date'}).text.replace('.','-').strip()
                    temp_value = r.find('td',{'class':'num'}).text.strip()
            
                    date.append(temp_date)
                    value.append(temp_value)
            else:

                temp = DataFrame(value, index = date, columns = [key])
                
                Data = pd.merge(Data,temp, how='outer', left_index=True, right_index=True)        
                
                print(key + '자료 수집 완료')
                time.sleep(10)
                break

    Data.to_csv('%s/silver_market_index.csv'%(folder_adress))
    return Data

In [10]:
def market_index_crawling():
    Data = DataFrame()
    
    url_dict = {'국제 백금':'https://finance.naver.com/marketindex/worldDailyQuote.nhn?marketindexCd=CMDT_PL&fdtc=2'}

    for key in url_dict.keys():
    
        date = []
        value = []

        for i in range(1,1000):
            url = re.get(url_dict[key] + '&page=%s'%i)
            url = url.content

            html = BeautifulSoup(url,'html.parser')

            tbody = html.find('tbody')
            tr = tbody.find_all('tr')
            
            
            '''마지막 페이지 까지 받기'''
            if len(tbody.text.strip()) > 3:
                
                for r in tr:
                    temp_date = r.find('td',{'class':'date'}).text.replace('.','-').strip()
                    temp_value = r.find('td',{'class':'num'}).text.strip()
            
                    date.append(temp_date)
                    value.append(temp_value)
            else:

                temp = DataFrame(value, index = date, columns = [key])
                
                Data = pd.merge(Data,temp, how='outer', left_index=True, right_index=True)        
                
                print(key + '자료 수집 완료')
                time.sleep(10)
                break

    Data.to_csv('%s/platinum_market_index.csv'%(folder_adress))
    return Data

In [3]:
def market_index_crawling():
    Data = DataFrame()
    
    url_dict = {'코스피':'https://finance.naver.com/sise/sise_index_day.nhn?code=KOSPI'}

    for key in url_dict.keys():
    
        date = []
        value = []

        for i in range(1,1000):
            url = re.get(url_dict[key] + '&page=%s'%i)
            url = url.content

            html = BeautifulSoup(url,'html.parser')

            tbody = html.find('tbody')
            tr = tbody.find_all('tr')
            
            
            '''마지막 페이지 까지 받기'''
            if len(tbody.text.strip()) > 3:
                
                for r in tr:
                    temp_date = r.find('td',{'class':'date'}).text.replace('.','-').strip()
                    temp_value = r.find('td',{'class':'num'}).text.strip()
            
                    date.append(temp_date)
                    value.append(temp_value)
            else:

                temp = DataFrame(value, index = date, columns = [key])
                
                Data = pd.merge(Data,temp, how='outer', left_index=True, right_index=True)        
                
                print(key + '자료 수집 완료')
                time.sleep(10)
                break

    Data.to_csv('%s/kospi_index.csv'%(folder_adress))
    return Data